In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

Here, we define the columns of the training set, and load it from the file.

In [3]:
trainingSetStamp='1561816373715'
training_column_names=["HAND_TYPE",
              "FINGER1_X", "FINGER1_Y", "FINGER1_Z",
              "FINGER2_X", "FINGER2_Y", "FINGER2_Z",
              "FINGER3_X", "FINGER3_Y", "FINGER3_Z",
              "FINGER4_X", "FINGER4_Y", "FINGER4_Z",
              "FINGER5_X", "FINGER5_Y", "FINGER5_Z"]
column_names=training_column_names[:]
column_names.append("INSTRUCTION_LABEL")
data = pd.read_csv('../../trainingAssets/sets/trainingSet_'+trainingSetStamp+'.txt', sep="|", header=None, names=column_names)

data

,HAND_TYPE,FINGER1_X,FINGER1_Y,FINGER1_Z,FINGER2_X,FINGER2_Y,FINGER2_Z,FINGER3_X,FINGER3_Y,FINGER3_Z,FINGER4_X,FINGER4_Y,FINGER4_Z,FINGER5_X,FINGER5_Y,FINGER5_Z,INSTRUCTION_LABEL
0,0,-120.6770,180.302,198.4600,-124.7150,181.188,122.7490,-145.4500,176.648,98.0581,-175.4580,169.432,92.2841,-229.2100,148.370,91.7343,0
1,0,-109.5860,218.408,171.7890,-124.5400,236.598,97.0678,-147.9920,236.790,74.3867,-178.0430,228.891,70.1260,-230.4720,204.815,70.1266,0
2,0,-102.1260,248.551,143.6590,-125.1300,261.200,67.2070,-150.3950,265.522,43.4693,-178.0430,228.891,70.1260,-230.4720,204.815,70.1266,0
3,0,-101.8930,249.117,143.3490,-125.1670,261.904,66.2591,-150.4780,266.152,42.7222,-179.0070,237.792,61.8757,-232.7410,221.071,58.9934,0
4,0,-96.9916,256.740,142.0780,-124.6310,264.666,61.8663,-150.2920,268.495,38.2543,-179.9050,250.776,47.2047,-234.4380,242.232,41.6788,0
5,0,-96.2495,258.979,142.2380,-124.4490,265.408,60.6466,-150.2180,269.029,37.0675,-180.0550,253.447,43.9323,-234.6170,245.644,38.5811,0
6,0,-95.1574,261.620,141.5660,-124.2120,266.095,59.4056,-150.1370,269.474,35.8924,-180.1490,255.479,41.1511,-234.6540,248.358,35.8936,0
7,0,-94.0546,264.116,140.3090,-123.9350,266.697,58.1889,-150.0460,269.849,34.7650,-180.1810,256.991,38.7966,-234.5550,250.445,33.4420,0
8,0,-92.9503,266.595,138.8810,-123.6520,267.217,57.0493,-149.9720,270.136,33.7769,-180.2010,258.084,36.8999,-234.4490,252.021,31.4099,0
9,0,-91.9838,268.778,137.0710,-123.3790,267.688,55.9700,-149.9150,270.386,32.9071,-180.2210,258.925,35.3368,-234.3510,253.279,29.6091,0


Now, we shuffle the data and partition it into training set and test set

In [4]:
data = data.sample(frac=1).reset_index(drop=True)
test_len = (len(data) * 20)//100;
training_data = data[test_len:]
test_data = data[:test_len]

Here we define the feature columns, describing the data on the data frame. 

TODO: Fix the type: The columns HAND_TYPE and INSTRUCTION_LABEL are int, not float32

In [5]:
feature_columns=[tf.contrib.layers.real_valued_column(column_name, dimension=1, dtype=tf.float32) for column_name in training_column_names]


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Format the data as required by Tensorflow

In [6]:
x_train_data = { column_name : np.array(training_data[column_name]) for column_name in training_column_names }

Define and train the model: A neural network classifier, with 5 layers with 5 neurons each, and a possible output of 5 labels.

In [9]:
classifier = tf.estimator.DNNClassifier(
       feature_columns = feature_columns,
       hidden_units = [5, 5],
       n_classes = 5)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = x_train_data,
    y = np.array(training_data["INSTRUCTION_LABEL"]).astype(int),
    num_epochs = None,
    shuffle = True)

# Train model.
classifier.train(input_fn = train_input_fn, steps = 2000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpv8uu8200', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2f93c52cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

Use the previously separated test data to evaluate the accuracy of the model

In [10]:
x_testing_data = { column_name : np.array(test_data[column_name]) for column_name in training_column_names }

# Define the training inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = x_testing_data,
    y = np.array(test_data["INSTRUCTION_LABEL"]).astype(int),
    num_epochs = 1,
    shuffle = False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("Test Accuracy: ", accuracy_score)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-29T14:08:00Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpv8uu8200/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-29-14:08:00
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.903, average_loss = 0.29695693, global_step = 2000, loss = 37.119617
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmpv8uu8200/model.ckpt-2000
Test Accuracy:  0.903


Export and store the trained model

In [11]:
def serving_input_receiver_fn():
    serialized_tf_example = tf.placeholder(dtype = tf.string, shape = [None], name = 'input_tensors')
    receiver_tensors      = {'predictor_inputs' : serialized_tf_example}
    feature_spec          = { column_name : tf.FixedLenFeature([25], tf.float32) for column_name in training_column_names }
    features              = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)


model_dir = classifier.export_savedmodel(export_dir_base = "../../trainingAssets/models/"+trainingSetStamp, 
                             serving_input_receiver_fn = serving_input_receiver_fn,
                             as_text = True)
print('Model exported to '+ model_dir.decode())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/tmpv8uu8200/model.ckpt-2000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../../trainingAssets/models/1561816373715/temp-b'1561817286'/saved_model.pbtxt
Model exported to ../../trainingAssets/models/1561816373715/1561817286
